In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../../data/'
group_dir_ = os.path.join('../../../../HPCF/all/test_0.8/data/groups/low')
folder = 'all (user)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [3]:
all_user = pd.read_pickle(os.path.join('../../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))
df = pd.read_pickle(os.path.join('../../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))

In [4]:
predictions[:5]

,uid,tid,rating
0,0,9,0.847839
1,0,15,0.594322
2,0,22,0.446187
3,0,28,0.721104
4,0,33,0.597911


In [5]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [6]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [7]:
num_user = len(test['uid'].unique())
num_user

953

In [8]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [9]:
top_n_size = 50
top_n_lists_g = []
top_n_items_g = []
for groups in groups_n:

    group_size = len(groups[0])
    top_n_lists = []
    top_n_items = pd.DataFrame()

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 
        
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)

        if len(top_n_items) == 0:
            top_n_items = rating_table
        else:
            top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

    top_n_lists_g.append(top_n_lists)   
    top_n_items_g.append(top_n_items)

COV = []    
NOV = []
HitNOV = []
HitCOV = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    top_n_items = top_n_items_g[i]

    nov = 0
    hitNov = 0
    hits = np.zeros(len(df['tid'].unique()), dtype=bool)
    for j in (range(len(groups))):
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        test_g = pd.DataFrame()
        for uid in group:
            t = test[test['uid'] == uid]
            if len(test_g) == 0:
                test_g = t
            else:
                test_g = test_g.set_index('tid').add(t.set_index('tid'), fill_value=0).reset_index()
        test_g['rating'] /= len(group)

        for k in range(top_n_size):
            tid = top_n_list[k]
            t = test_g[test_g['tid'] == tid]
            nov += (1-pop_dict[tid])
            if len(t) > 0:
                hits[int(tid)] = True
                hitNov += (1-pop_dict[tid])
    NOV.append(nov/top_n_size/(len(groups)))
    COV.append(len(top_n_items)/len(df['tid'].unique()))
    HitNOV.append(hitNov/top_n_size/(len(groups)))
    HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
print(NOV)
print(HitNOV)
print(COV)
print(HitCOV)

[0.5981004147857896, 0.607602869455612, 0.6150697201458235, 0.6100769499825134, 0.6014537425673314, 0.5989447628272375, 0.5928551602748517]
[0.16115378466600597, 0.1813839117780993, 0.1910117804870127, 0.20004292664313633, 0.196033140958377, 0.1947835445422014, 0.2037051078089564]
[0.0015739336282343385, 0.0015422010954070332, 0.0015041220560142669, 0.0013708454181395851, 0.0011360246752175264, 0.0011106386489556823, 0.0010915991292592992]
[0.0014850825363178837, 0.0014470034969251175, 0.0013898849378359683, 0.001205836247437598, 0.0010281340636046888, 0.0009900550242119225, 0.00091389694542639]
